In [96]:
%cd "drive/My Drive/VQA-master"

[Errno 2] No such file or directory: 'drive/My Drive/VQA-master'
/content/drive/My Drive/VQA-master


In [0]:
import json 
  
# JSON file
def read_json(filename): 
  f = open (filename, "r") 
    
  # Reading from file 
  data = json.loads(f.read()) 
  return data

In [0]:
train_data = read_json("train_dataset_5k_tokenised.json")
val_data = read_json("val_dataset_1k_tokenised.json")

In [0]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

In [0]:
!unzip glove.6B.zip

In [100]:
!pip install bcolz

     |████████████████████████████████| 1.5MB 7.3MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2662291 sha256=56c7035b501b7512f1b4f09f9628ebcd6e269c92aaf872dd5df1a35afc0eb037
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [0]:
import bcolz
import numpy as np
import pickle

words = []
idx = 0
word2idx = {}
glove_path = ""
vectors = bcolz.carray(np.zeros(1), rootdir=f'6B.50.dat', mode='w')

with open(f'glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400001, 50)), rootdir=f'6B.50.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'6B.50_idx.pkl', 'wb'))

In [0]:
vectors = bcolz.open(f'6B.50.dat')[:]
words = pickle.load(open(f'6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'6B.50_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [0]:
matrix_len = 400001
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(glove.keys()):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

In [0]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

class ToyNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers):
        super(ToyNN, self).__init__()
        print(weights_matrix.size())
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp))
    
    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [0]:
# image_features = np.load("train_image_features.npy")
# questions = np.load("train_questions.npy")
# answers = np.load("train_answers.npy")
# combined_train_input = np.load("combined_input.npy")

image_features = np.load("val_image_features.npy")
questions = np.load("val_questions.npy")
val_answers = np.load("val_answers.npy")
# combined_val_input = np.load("combined_val_input.npy")

In [107]:
import torch
weights_matrix = torch.tensor(weights_matrix)
lstm = ToyNN(weights_matrix, 2048, 1)
questions = torch.tensor(questions, dtype=torch.long)
# image_features = torch.tensor(image_features, dtype=torch.float)
# image_features = image_features.view(1, image_features.size(0), image_features.size(1))
output = lstm(questions, image_features)
# print(output.size())

torch.Size([400001, 50])


In [108]:
a,b = output
b = b.view(b.size(1), b.size(2))
image_features = torch.tensor(image_features, dtype=torch.float)
print(image_features.size())
print(b.size())

torch.Size([837, 512])
torch.Size([837, 2048])


In [109]:
combined_input = torch.cat((image_features,b), 1)
print(combined_input.size())

torch.Size([837, 2560])


In [0]:
combined_val_input = combined_input.detach().numpy()
np.save("combined_val_input.npy", combined_val_input)

In [0]:
import numpy as np

In [0]:
# image_features = np.load("train_image_features.npy")
# questions = np.load("train_questions.npy")
train_answers = np.load("train_answers.npy")
combined_train_input = np.load("combined_train_input.npy")

In [0]:
# image_features = np.load("val_image_features.npy")
# questions = np.load("val_questions.npy")
val_answers = np.load("val_answers.npy")
combined_val_input = np.load("combined_val_input.npy")

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2560, 1000)
        # self.fc2 = nn.Linear(2000, 1500)
        # self.fc3 = nn.Linear(1500, 1000)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.softmax(x)
        return x



In [117]:
import torch
from torch.utils import data
tensor_x = torch.Tensor(combined_train_input)
tensor_y = torch.Tensor(train_answers).flatten()
tensor_y = tensor_y.type(torch.long)
print(tensor_x.size())
print(tensor_y.size())
print(tensor_x.dtype)
print(tensor_y.dtype)
trainset = data.TensorDataset(tensor_x,tensor_y)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True, num_workers=2)

torch.Size([4269, 2560])
torch.Size([4269])
torch.float32
torch.int64


In [123]:
from torch.utils import data
tensor_x = torch.Tensor(combined_val_input)
tensor_y = torch.Tensor(val_answers).flatten()
tensor_y = tensor_y.type(torch.long)
print(tensor_x.size())
print(tensor_y.size())
print(tensor_x.dtype)
print(tensor_y.dtype)
valset = data.TensorDataset(tensor_x,tensor_y)
valloader = torch.utils.data.DataLoader(valset, batch_size=1000, shuffle=True, num_workers=2)

torch.Size([837, 2560])
torch.Size([837])
torch.float32
torch.int64


In [0]:
net = Net()
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [0]:
def get_accuracy(predictions, labels):
  predictions = torch.max(predictions, axis=1)[1]
  # print(predictions)
  # print(labels)
  # print(predictions.size())
  # print(labels.size())
  ab = torch.abs(predictions-labels)
  # print(ab.size())
  ab = ab.detach().numpy()
  mn = np.minimum(ab, 1)
  eq = 1-mn
  # eq = 1 - torch.min(torch.abs(predictions-labels), 1)
  correct = np.sum(eq)
  # print(correct)
  total = eq.shape[0]
  return correct, total

In [133]:
for epoch in range(500):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    total_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        batch_correct, batch_total = get_accuracy(outputs, labels)
        correct += batch_correct
        total += batch_total
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 1 == 0:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
        total_loss += running_loss
        running_loss = 0.0

    val_correct = 0
    val_total = 0
    net.eval()
    with torch.no_grad():
      for i, data in enumerate(valloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # forward + backward + optimize
          outputs = net(inputs)
          batch_correct, batch_total = get_accuracy(outputs, labels)
          val_correct += batch_correct
          val_total += batch_total
          
    print("Epoch: ",epoch," Loss: ",total_loss," Train-Accuracy: ", correct/total," Val-Accuracy: ",val_correct/val_total)
    

print('Finished Training')

Epoch:  0  Loss:  34.53112554550171  Train-Accuracy:  0.015928788943546497  Val-Accuracy:  0.03464755077658303
Epoch:  1  Loss:  34.46015644073486  Train-Accuracy:  0.06371515577418599  Val-Accuracy:  0.11947431302270012
Epoch:  2  Loss:  34.114097595214844  Train-Accuracy:  0.2101194659170766  Val-Accuracy:  0.2078853046594982
Epoch:  3  Loss:  33.54840564727783  Train-Accuracy:  0.23518388381353947  Val-Accuracy:  0.23178016726403824
Epoch:  4  Loss:  33.43857955932617  Train-Accuracy:  0.23284141485125323  Val-Accuracy:  0.23536439665471923
Epoch:  5  Loss:  33.388670444488525  Train-Accuracy:  0.23213867416256734  Val-Accuracy:  0.23297491039426524
Epoch:  6  Loss:  33.37896728515625  Train-Accuracy:  0.23167018037011008  Val-Accuracy:  0.23297491039426524
Epoch:  7  Loss:  33.38943529129028  Train-Accuracy:  0.23167018037011008  Val-Accuracy:  0.23297491039426524
Epoch:  8  Loss:  33.36867618560791  Train-Accuracy:  0.23167018037011008  Val-Accuracy:  0.23297491039426524
Epoch:  9